+1 57166..61908  4743 <br>
...

# Extra Credit - 10 points possible
 - Allow your program to optionally use a set of start codons - ATG, GTG, TTG for example.
 - Allow your program to optionally use a set of stop codons - TAG, TGA, TAA for example.
 - Allow your program to print every putative gene in an ORF instead of only the largest.
 - Allow your program to have varying ORF size minimums 

# Submit your code and answers
For this lab, you should upload the following three files as attachments:

 - your sequenceAnalysis.py module
 - findORFs.py, include design in # comments at the beginning of findORFs.py
 - tass2ORFdata-ATG-100.txt

Important: to get full credit on this lab assignment, each of the code files you submit needs to:
 - Run properly (execute and produce the correct output)
 - Include an overview about what your program is designed to do with expected inputs and outputs
 - Include design (place at the top of your file below your name, group, and description). Include any assumptions or design decisions you made in writing your code
 - Adhere to the Report format specification
 - Include docstrings for Program, Module, Class, Methods and Functions
 - Contain in-line comments
 
 Congratulations, you finished your fifth lab assignment!

# Hints:
## One strand at a time

One design solution considers only the top strand to generate gene candidates. We then generate the reverse complement and a second list of gene candidates.  When done in this way, remember to fix the start/end coordinates that you get from the reverse complement solution, since those will be based on the other end of the sequence. 

An alternative design exists for the reverse strand that does not require generation of the reverse complement. This solution is interesting and may be simpler to consider. This solution scans the bottom strand left to right, finding stop codons first (reverse complement), then looking for start codons (reverse complement).

A third solution for the bottom strand would scan right to left, finding starts until a stop is encountered.

## Start and Stop handling

If you are going to do the extra-credit, then you will need to remember where the starts are until you find a stop.  This is straightforward if you place those start positions in a list, organized by frame. Even if you are not doing the extra-credit, placing starts on a list organized by frame will really simplify your code. This solution would require three lists for each of the three reading frames.

As an alternative, you could scan each of the three reading frames seperately, counting by threes in each case. This seems to be the most straightforward way to do this assignment.

When you do find a stop, you then consider the start(s) that you found along the way in this ORF.  Notice that the "longest" ORF is then defined by the start that is at the beginning of your start list.  Don't forget to clean up this list after you find genes using this stop since it would terminate all of those "pending" genes. What would happen if you didn't clean up your start list - you would be finding ORF candidates that had a stop in the middle of them, which means they are not __OPEN__ reading frames.

## Termini and gene fragments

This is a bit easier if you think that position 0(1 externally) will allways be a start position for a gene in every frame. This is the case where a start codon exists somewhere upstream of our sequence. It would code for a gene that would terminate in our sequence somewhere. This is the classic example of a dangling-stop. We would report this gene fragment beginning at position 0(1) and ending at the end of that in-frame stop. In frame 1, there might actually be a start codon at this position so dont count it twice. In Frames 2 or 3, there are extra bases ahead of our first full codon, so we will always have the possibility that the actual start is upstream of this dna fragment, so.. we report this as a dangling stop case starting at position 0(1).  

Dangling starts always end at the end of sequence, even if that position is out of frame. In cases where no start or stop codon is found anywhere in the frame, the entire sequence is then considered as a gene.

# Data and output files

The cell below is set up for the non-extra credit version of the program with a fixed filename. Your much more usable program will use STDIN and STDOUT in command line fashion. To do this, simplify main so your fastaReader object is reading from STDIN.

All of the input files and programs should be in the same folder so you won’t have to specify a path, if your data file is in some other folder, you can provide that like this:<br> ~/Desktop/bme160/someOtherPlace/labData.fa.

# Comments

To get full credit for this lab, you must turn in your design for each program you submit. Remember that any item you write or include must have proper docstrings.

In [ ]:
#!/usr/bin/env python3
# Name: Brandon Wong (bwong44)
# Group Members: Tim Lee, Vibha Rao, Penetha Jayakumar

########################################################################
# CommandLine
########################################################################
class CommandLine() :
    '''
    Handle the command line, usage and help requests.

    CommandLine uses argparse, now standard in 2.7 and beyond. 
    it implements a standard command line argument parser with various argument options,
    a standard usage and help.

    attributes:
    all arguments received from the commandline using .add_argument will be
    avalable within the .args attribute of object instantiated from CommandLine.
    For example, if myCommandLine is an object of the class, and requiredbool was
    set as an option using add_argument, then myCommandLine.args.requiredbool will
    name that option.
 
    '''
    
    def __init__(self, inOpts=None) :
        '''
        Implement a parser to interpret the command line argv string using argparse.
        '''
        
        import argparse
        self.parser = argparse.ArgumentParser(description = 'Program prolog - a brief description of what this thing does', 
                                             epilog = 'Program epilog - some other stuff you feel compelled to say', 
                                             add_help = True, #default is True 
                                             prefix_chars = '-', 
                                             usage = '%(prog)s [options] -option1[default] <input >output'
                                             )
        self.parser.add_argument('-lG', '--longestGene', action = 'store', nargs='?', const=True, default=False, help='longest Gene in an ORF')
        self.parser.add_argument('-mG', '--minGene', type=int, choices= (100,200,300,500,1000), default=100, action = 'store', help='minimum Gene length')
        self.parser.add_argument('-s', '--start', action = 'append', default = ['ATG'],nargs='?', 
                                 help='start Codon') #allows multiple list options
        self.parser.add_argument('-t', '--stop', action = 'append', default = ['TAG','TGA','TAA'],nargs='?', help='stop Codon') #allows multiple list options
        self.parser.add_argument('-v', '--version', action='version', version='%(prog)s 0.1')  
        if inOpts is None :
            self.args = self.parser.parse_args()
        else :
            self.args = self.parser.parse_args(inOpts)

########################################################################
# OrfFinder
# Here is the OrfFinder program
# 
#
########################################################################

class OrfFiner:
    """
    Defines a class that reads the sequence and finds the open reading frames.

    Iterates 3 times for each frame
        Iterates for each codon stepping by 3 characters
            If has start codon:
                Append to start codon list
            If has stop codon:
                For items in start codon list:
                    Appends to geneFragList (frame,start,stop,length)
                Clears start codon list
            If there has been no start codons but found a stop codon we have found a dangling stop:
                Appends to geneFragList(frame,start,stop,length)
                Clears start codon list
        If has start but no stop we have found a dangling start:
            Appends to geneFragList(frame,start,stop,length)
        If doesn't have start, stop we have found a dangling start stop:
            Append to geneFragList(frame,start,stop,length)
        adjustFrame += 1
    
    """
    def __init__(self):
        """Initializes the OrfFinder class and creates dictionaries that stores the ORF data."""
        self.startSet = set()
        self.stopSet = set()
        self.orfList = [] #Holds start positions
        self.geneFragList = [] #[reading frame, start, end, length]
        self.minOrf = 100
        self.longestGene = False

    def readFrame(self, seq):
        """Accepts the sequence as a string from fasta and parses it for the ORFs, looking for gene fragments."""
        adjustFrame = 0 #Stores value of which reading frame it is in 'TAG','TGA','TAA
        danglingStartStop = []
        hasStart = 0
        hasCodon = 0
        for orf in range(3): #Iterates for the 3 reading frames
            frameOne = range(adjustFrame,len(seq),3)
            hasStart = 0
            hasStop  = 0
            self.orfList.clear()
            for frame in frameOne: #Iterates through the sequence 3 codons at a time
                if seq[frame:frame+3] in self.startSet: #Checks for start
                    self.orfList.append(frame + 1)
                    hasStart = 1

                elif seq[frame:frame+3] in self.stopSet and hasStart: #Checkts for stop
                    for pos in self.orfList:
                        self.geneFragList.append([adjustFrame + 1,pos,frame+3,frame+3 - pos + 1])
                    hasStop = 1
                    self.orfList.clear()

                elif hasStart == 0 and seq[frame:frame+3] in self.stopSet: #Checks for dangling stop (we find stop but no start)
                    self.geneFragList.append([adjustFrame+1,1,frame+3,frame+3])
                    hasStop = 1
                    self.orfList.clear()

            if hasStart == 1 and hasStop == 0: #Checks for dangling start (we find start but no stop)
                self.geneFragList.append([adjustFrame+1,1,len(seq),len(seq)])
            elif hasStart == 0 and hasStop == 0 and self.orfList == []: #Checks for dangling start and stop 
                self.geneFragList.append([adjustFrame+1,1,len(seq),len(seq)])
            adjustFrame +=1                 

    def readReverseComp(self, seq):
        """Takes the reverse complement of the sequuence and searches for valid gene fragments"""
        complementDict = {"A":"T","T":"A","G":"C","C":"G",}
        seqReverse = seq[::-1]
        reverseComplement = "".join(complementDict[nucleotide] for nucleotide in seqReverse)
        adjustFrame = 0 #Stores value of which reading frame it is in 'TAG','TGA','TAA
        negativeFrame = 0 #Stores negative frame value
        hasStart = 0
        hasCodon = 0
        for orf in range(3): #Iterates for the 3 reading frames
            frameOne = range(adjustFrame,len(reverseComplement),3)
            hasStart = 0
            hasStop  = 0
            self.orfList.clear()
            for frame in frameOne: #Iterates through the sequence 3 codons at a time
                if reverseComplement[frame:frame+3] in self.startSet: #Checks for start
                    self.orfList.append(frame + 1)
                    hasStart = 1

                elif reverseComplement[frame:frame+3] in self.stopSet and hasStart: #Checkts for stop
                    for pos in self.orfList:
                        self.geneFragList.append([negativeFrame-1,len(reverseComplement)-pos+1,len(reverseComplement)-frame-2,frame+3 - pos + 1])
                    hasStop = 1
                    self.orfList.clear()

                elif hasStart == 0 and reverseComplement[frame:frame+3] in self.stopSet: #Checks for dangling stop (we find stop but no start)
                    self.geneFragList.append([negativeFrame-1,len(reverseComplement),len(reverseComplement)-frame-2,frame+3])
                    hasStop = 1
                    self.orfList.clear()

            if hasStart == 1 and hasStop == 0: #Checks for dangling start (we find start but no stop)
                self.geneFragList.append([negativeFrame-1,1,len(reverseComplement),len(reverseComplement)])
            elif hasStart == 0 and hasStop == 0 and self.orfList == []: #Checks for dangling start and stop 
                self.geneFragList.append([negativeFrame-1,1,len(reverseComplement),len(reverseComplement)])
            adjustFrame +=1
            negativeFrame -= 1
        
    def minimumOrf(self):
        "Excludes the genes with the length < minOrf"
        minLenList = []
        for gene in range(len(self.geneFragList)): #Iterates through the list of gene fragments
            if self.geneFragList[gene][3] > self.minOrf: #Checks if the gene fragment is greater than the min
                minLenList.append(self.geneFragList[gene])
        self.geneFragList = minLenList

    def sortBySize(self):
        """Takes the list of gene fragments and sorts by the largest gene size then starting number."""
        sortedGeneList = sorted(self.geneFragList, key = lambda x: (-x[-1],x[1])) #Sorts by gene length
        self.geneFragList = sortedGeneList

    def largestOrf(self):
        "Returns only the largest genes if the option is set."
        if self.longestGene == True: #Checks if longest gene is written in terminal
            largestList = []
            compareSet = set()
            for gene in self.geneFragList: #Compares the ending position since it is already sorted by largest gene
                if gene[2] not in compareSet:
                    largestList.append(gene)
                    compareSet.add(gene[2])
            self.geneFragList = largestList
        
    def reverseOrder(self):
        """Reverses the order so that the start and stop match the top strand."""
        for gene in self.geneFragList:
            if gene[0] < 0:
                startHolder = gene[1]
                stopHolder = gene[2]
                gene[1] = stopHolder
                gene[2] = startHolder
            
    def clearGeneFragList(self):
        """Clears out the geneFragList object so that it doesn't store the data of previous sequences if there are any."""
        self.geneFragList = []

########################################################################
# Main
# Here is the main program
# 
#
########################################################################
   
def main(inFile = None, options = None):
    """Finds all valid gene fragments using the terminal to specify options."""
    thisCommandLine = CommandLine(options)
    reader = FastaReader(inFile)
    
if __name__ == "__main__":
    main()


In [ ]:
python3 findOrfs.py  <somefile  >someOtherFile -mG 300 -lG -s ATG -s GTG

# findORFs output using tass2.fa  -mG=300 -lG

In [ ]:
tass2 NODE_159_length_75728_cov_97.549133
+1 57166..61908  4743
-1  8192..11422  3231
+2 65963..69004  3042
-3 14589..16862  2274
-2  2968.. 4872  1905
+1 64093..65952  1860
-3    30.. 1694  1665
+1 69475..71052  1578
+1 48805..50223  1419
+1 47398..48798  1401
-3 29133..30500  1368
-1 40922..42250  1329
-2 19975..21270  1296
+3 72273..73559  1287
-1 24482..25639  1158
-1  6689.. 7804  1116
+1 50251..51366  1116
-3 27501..28601  1101
+2 63038..64078  1041
+3 62019..63038  1020
-2 42271..43263   993
+3 51864..52805   942
+1 45484..46371   888
-3 11433..12317   885
+2 74357..75184   828
+2 71576..72394   819
+3 46341..47138   798
-2 18613..19407   795
+1 55642..56388   747
-1 16940..17632   693
-2 13288..13974   687
-3 26115..26801   687
-1 21338..21994   657
-1 30998..31654   657
-2 12601..13251   651
-2  4894.. 5532   639
-3 32592..33221   630
-1 39914..40525   612
+1 53977..54588   612
+3 54588..55193   606
-3 33234..33809   576
-3 22002..22559   558
-3 23859..24413   555
-2  1945.. 2490   546
+1 73861..74370   510
-2  6214.. 6696   483
+1 16324..16806   483
-1 22556..23032   477
+3 32808..33260   453
+2 53324..53776   453
+1 29056..29502   447
-2 36286..36729   444
+3 51396..51833   438
+2 55196..55627   432
-1  2468.. 2896   429
-2 31798..32220   423
+1 52891..53307   417
-2 30595..30996   402
-2  5809.. 6204   396
+1     1..  393   393
-2 35740..36129   390
-3 34542..34925   384
+2 56474..56857   384
-1 17888..18268   381
-3 23004..23381   378
-3 52968..53336   369
-1 32207..32572   366
-1 23396..23752   357
+2 36419..36775   357
-2 18268..18609   342
+2 44594..44935   342
+1 43714..44049   336
+2  6557.. 6886   330
-2 34927..35253   327
-3 34131..34451   321
+1 30778..31095   318
+2 24974..25288   315
+1  1246.. 1557   312
-3 35253..35561   309
-2 39160..39468   309
-2 68932..69234   303

# findORFs output using lab5test.fa at mG=0 cutoff

(Note: I have included the sequence of the gene candidate as an additional column)

In [ ]:
test
+1     1..    9     9 ATGATGTAA
+3     1..    9     9 AT GAT GTA A
-1     1..    9     9 TTACATCAT
-2     1..    9     9 T TAC ATC AT
-3     1..    9     9 TTACATCAT
+1     4..    9     6 ATG TAA
+2     1..    4     4 ATGA
test2
+1     1..   10    10 AA TGA TGT AA
+2     1..   10    10 A ATG ATG TAA
-1     1..   10    10 TTACATCATT
-2     1..   10    10 TTACATCATT
-3     1..   10    10 TTACATCATT
+2     2..   10     9 ATGATGTAA
+2     5..   10     6 ATGTAA
+3     1..    5     5 AATGA
test3
+2     1..   11    11 A AAT GAT GTA A
+3     1..   11    11 AA ATG ATG TAA
-1     1..   11    11 TTACATCATTT
-2     1..   11    11 TTACATCATTT
-3     1..   11    11 TTACATCATTT
+3     3..   11     9 ATG ATG TAA
+1     1..    6     6 AAATGA
+3     6..   11     6 ATGTAA
test-1
+1     1..    9     9 TTACATCAT
+2     1..    9     9 TTACATCAT
+3     1..    9     9 TTACATCAT
-1     1..    9     9 ATGATGTAA
-3     1..    9     9 ATGATGTAA
-1     1..    6     6 ATGTAA
-2     6..    9     4 ATGA
test-2
+1     1..   10    10 TTACATCATA
+2     1..   10    10 TTACATCATA
+3     1..   10    10 TTACATCATA
-1     1..   10    10 TATGATGTAA
-2     1..   10    10 TATGATGTAA
-2     1..    9     9 ATGATGTAA
-2     1..    6     6 ATGTAA
-3     6..   10     5 TATGA
test-3
+1     1..   11    11 TTACATCATAA
+2     1..   11    11 TTACATCATAA
+3     1..   11    11 TTACATCATAA
-2     1..   11    11 TTATGATGTAA
-3     1..   11    11 TTATGATGTAA
-3     1..    9     9 ATGATGTAA
-3     1..    6     6 ATGTAA
-1     6..   11     6 TTATGA
test1A
+3     1..   10    10 ATGATGTAAA
-1     1..   10    10 TTTACATCAT
-2     1..   10    10 TTTACATCAT
-3     1..   10    10 TTTACATCAT
+1     1..    9     9 ATGATGTAA
+1     4..    9     6 ATGTAA
+2     1..    4     4 ATGA
test2A
+1     1..   11    11 AATGATGTAAA
-1     1..   11    11 TTTACATCATT
-2     1..   11    11 TTTACATCATT
-3     1..   11    11 TTTACATCATT
+2     1..   10    10 AATGATGTAA
+2     2..   10     9 ATGATGTAA
+2     5..   10     6 ATGTAA
+3     1..    5     5 AATGA
test3A
+2     1..   12    12 AAATGATGTAAA
-1     1..   12    12 TTTACATCATTT
-2     1..   12    12 TTTACATCATTT
-3     1..   12    12 TTTACATCATTT
+3     1..   11    11 AAATGATGTAA
+3     3..   11     9 ATGATGTAA
+1     1..    6     6 AAATGA
+3     6..   11     6 ATGTAA
test-1A
+1     1..   10    10 ATTACATCAT
+2     1..   10    10 ATTACATCAT
+3     1..   10    10 ATTACATCAT
-3     1..   10    10 ATGATGTAAT
-1     2..   10     9 ATGATGTAA
-1     2..    7     6 ATGTAA
-2     7..   10     4 ATGA
test-2A
+1     1..   12    12 AATTACATCATA
+2     1..   12    12 AATTACATCATA
+3     1..   12    12 AATTACATCATA
-1     1..   12    12 TATGATGTAATT
-2     3..   12    10 TATGATGTAA
-2     3..   11     9 ATGATGTAA
-2     3..    8     6 ATGTAA
-3     8..   12     5 TATGA
test-3A
+1     1..   13    13 AATTACATCATAA
+2     1..   13    13 AATTACATCATAA
+3     1..   13    13 AATTACATCATAA
-2     1..   13    13 TTATGATGTAATT
-3     3..   13    11 TTATGATGTAA
-3     3..   11     9 ATGATGTAA
-3     3..    8     6 ATGTAA
-1     8..   13     6 TTATGA

In [ ]:
#Inspection Results:

#Vibha inspection:
I have no comments. The code looks great! I like how you've implemented the longestOrf method. 
What does the reverseOrder method do?

#Tim Inpsection:
You have some extra commented code that you can clean up.

#Response:
For my reverseOrder method it iterates through all the gene fragments and searches for the negative frames. Once it finds a negative frame it changes the start and stop position to the correct 
position to be in the same orientation as the top strand.

I took out the exta lines to clean up my code.